# Process the restaurant data

Process the restaurant data such that it can be used for different kinds of analyses

The atomic unit of the analysis is an activity which has the following properties:
1. Start time [hh:mm;ss]
2. End time [hh:mm;ss]
3. Location ID
4. Staff ID (only relevant for staff activities)
5. Activity type







The resulting data sets are:
1. The sequence of activities performed by a staff member per continuous observation time frame 
2. The sequence of activities performed per table per continuous observation time frame and per group visit
3. Demand pattern per table
4. Toilet visits


In [1]:
from pathlib import Path
import pandas as pd


## Clean and save data in pandas dataframe format

### Staff activities - evening 18-08-2021

In [9]:
filename = Path('./data/raw/20210818E1_AM-ZZ.xlsm')
df_staff_actvities_1 = pd.read_excel(filename, sheet_name='Staff table visit (Staff)', header=10, skiprows=[11,101], usecols='B:F', engine='openpyxl')

filename = Path('./data/raw/20210818E2_AM-ZZ.xlsm')
df_staff_actvities_2 = pd.read_excel(filename, sheet_name='Staff table visit (Staff)', header=10, skiprows=[11,123], usecols='B:F', engine='openpyxl')

df_staff_actvities = pd.concat([df_staff_actvities_1, df_staff_actvities_2], ignore_index=True)
df_staff_actvities.rename(columns={'Table ID': 'table_id', 'Staff ID (names or name initials)': 'staff_id',
                                  'Arrival time': 'start_time', 'Departure time': 'end_time',
                                  'Visit type (Taking order - O/ Serving order-S/ Check-out&Pay - P/ Cleaning - C / checking for feedback|tasting - F)': 'activity_type'}, inplace=True)

out_filename = Path('./data/processed/20210818_evening_staff_activities.csv')
df_staff_actvities.to_csv(out_filename, index=False)

### Guest demand - evening 18-08-2021

In [3]:
filename = Path('./data/raw/20210818_1710_MS_SSO_DataForCalibration.xlsm')
df_demand_raw = pd.read_excel(filename, sheet_name='Customer demand (all the time)', header=21, usecols='B:G', engine='openpyxl')

from extract_distribution import extract_visits_per_table
df_demand = extract_visits_per_table(df_demand_raw)

out_filename = Path('./data/processed/20210818_evening_guest_demand.csv')
df_demand.to_csv(out_filename, index=False)

### Toilet visits - evening 18-08-2021

In [27]:
filename = Path('./data/raw/20210818_1710_MS_SSO_DataForCalibration.xlsm')
df_toilet_raw = pd.read_excel(filename, sheet_name='Toilet visit', header=7, usecols='B:D', engine='openpyxl')

df_toilet = df_toilet_raw.drop(df_toilet_raw[df_toilet_raw['Type (Guest - G or Staff - S)'].isnull()].index)
df_toilet.rename(columns={'Type (Guest - G or Staff - S)': 'guest_or_staff',
                                  'Arrival time': 'start_time', 'Departure time': 'end_time'}, inplace=True)


import numpy as np
import datetime

df_toilet['start_time'] = df_toilet['start_time'].apply(lambda value: value if isinstance(value, datetime.datetime) else np.nan)
df_toilet['end_time'] = df_toilet['end_time'].apply(lambda value: value if isinstance(value, datetime.datetime) else np.nan)
df_toilet['visit_time'] = df_toilet.apply(lambda row: (row['end_time'] - row['start_time'])/np.timedelta64(1, 's') if pd.notnull(row['start_time']) and pd.notnull(row['end_time']) else np.nan, axis=1)

df_toilet[df_toilet['visit_time'].isnull()]

out_filename = Path('./data/processed/20210818_evening_toilet_visits.csv')
df_toilet.to_csv(out_filename, index=False)

0      19.0
1       9.0
2       6.0
3       3.0
4       3.0
       ... 
387     8.0
388    17.0
389     5.0
390     6.0
391     8.0
Length: 392, dtype: float64

170    76.0
171    10.0
172     6.0
173    26.0
174    24.0
175    12.0
176    32.0
177     8.0
178    11.0
179    12.0
dtype: float64